In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import json

In [3]:
RESULTS_ROOT = './results/models'
model_name = "LESSVIT"

In [4]:
# for dir in os.listdir(os.path.join(RESULTS_ROOT, "bigearthnet")):
#     model_name, embed_dims, depth, _, _, lr, scale = tuple(dir.split("_"))
#     new_dir = f"{model_name}_{embed_dims}_{depth}_bigearthnet_lr{lr}_{scale}_moe0"
#     # print(new_dir)
#     os.rename(os.path.join(RESULTS_ROOT, "bigearthnet", dir), os.path.join(RESULTS_ROOT, "bigearthnet", new_dir))

In [9]:
def collect_results(model_name, dataset, metric_name):
    dataset_dir = f"{RESULTS_ROOT}/{dataset}"
    
    df_all = []
    for dir in os.listdir(dataset_dir):
        target_file = os.path.join(dataset_dir, dir, "test_results.json")
        try:
            with open(target_file, 'r') as f:
                log = json.load(f)
        except:
            continue
        model_config = dir.split("_")
        _, embed_dims, depth, _, lr, scale, moe = tuple(model_config)
        log['embed_dims'] = int(embed_dims.replace("b", ""))
        log['depth'] = int(depth.replace("d", ""))
        log['lr'] = float(lr.replace("lr", ""))
        log['scale'] = float(scale.replace("scale", ""))
        log['moe'] = int(moe.replace("moe", ""))
        # log['model_name'] = dir
        df_all.append(log)
            
    df_all = pd.DataFrame(df_all)
    
    df_all_stat = df_all.groupby(['embed_dims', 'depth', 'scale', 'moe']).apply(lambda x: x.loc[x[metric_name].idxmax()])
    df_all_stat = df_all_stat.drop(columns=['embed_dims', 'depth', 'scale', 'moe'])
    df_all_stat = df_all_stat.reset_index()
    df_all_stat.sort_values(by=[metric_name], ascending=False)
    return df_all, df_all_stat

In [6]:
dataset = 'eurosat'
metric_name = 'eval_accuracy'

df_all, df_all_stat = collect_results(model_name, dataset, metric_name)
df_all_stat

,embed_dims,depth,scale,moe,epoch,eval_accuracy,eval_loss,eval_runtime,eval_samples_per_second,eval_steps_per_second,lr
0,2,4,1.0,0,20.0,0.971667,0.138264,11.1384,484.809,3.861,0.00005
1,2,8,1.0,0,20.0,0.971481,0.139891,15.5711,346.796,2.762,0.00010


In [7]:
dataset = 'bigearthnet'
metric_name = 'eval_micro_mAP'

df_all, df_all_stat = collect_results(model_name, dataset, metric_name)
df_all_stat

,embed_dims,depth,scale,moe,epoch,eval_loss,eval_micro_mAP,eval_runtime,eval_samples_per_second,eval_steps_per_second,lr
0,2,4,1.0,0,9.909953,0.167150,0.856657,915.4190,137.496,1.075,0.00008
1,2,8,1.0,0,9.909953,0.168938,0.857083,1063.7797,118.320,0.925,0.00030


In [10]:

dataset = 'segmunich'
metric_name = 'eval_IoU'

df_all, df_all_stat = collect_results(model_name, dataset, metric_name)
df_all_stat

1,embed_dims,depth,scale,moe,epoch,eval_IoU,eval_loss,eval_runtime,eval_samples_per_second,eval_steps_per_second,lr
0,2,4,1.0,0,9.994595,0.404518,0.730009,151.9596,64.794,0.507,0.00005
